# Labo 6

Lucas Charbonnier & Rémi Ançay

In [3]:
from nltk.corpus import reuters
import nltk

nltk.download('reuters')

print('training files : ', len([fid for fid in reuters.fileids() if fid[:5] == 'train']))
print('testing files : ', len([fid for fid in reuters.fileids() if fid[:4] == 'test']))
print('total files : ', len(reuters.fileids()))

training files :  7769
testing files :  3019
total files :  10788


[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\lcsch\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


## Question 1

Il y a 10788 documents au total, dont 7769 d'entraînement et 3019 de test. Nous rechercherons les paramètres optimaux sur les documents de test.

## Question 2
Transformation en dataframes pandas

In [4]:
import pandas as pd

# On récupère tous les ids des fichiers qui nous intéressent et on les sépare en deux catégories; train et test
train_fids = [fid for fid in reuters.fileids() if fid[:5] == 'train']
test_fids = [fid for fid in reuters.fileids() if fid[:4] == 'test']

# Fonction permettant d'obtenir le texte complet ainsi que la catégorie (1 si grain) du fichier avec l'id spécifié
def get_text_and_category(fileid):
    fulltext = ' '.join(reuters.words(fileid))
    category = 1 if 'grain' in reuters.categories(fileid) else 0
    return fulltext, category

# Regroupement des données
train_data = [(get_text_and_category(fileid)) for fileid in train_fids]
train_df = pd.DataFrame(train_data, columns=['fulltext', 'category'])

# Création des dataframes pandas
test_data = [(get_text_and_category(fileid)) for fileid in test_fids]
test_df = pd.DataFrame(test_data, columns=['fulltext', 'category'])

train_df.head()
test_df.head()

,fulltext,category
0,ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...,0
1,CHINA DAILY SAYS VERMIN EAT 7 - 12 PCT GRAIN S...,1
2,JAPAN TO REVISE LONG - TERM ENERGY DEMAND DOWN...,0
3,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER Tha...,1
4,INDONESIA SEES CPO PRICE RISING SHARPLY Indone...,0


## Question 3

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from pprint import pprint
from sklearn.model_selection import GridSearchCV
from time import time
import numpy as np

pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("clf", SGDClassifier(loss="hinge")),
    ]
)

# Grille de recherche des paramètres à tester
# Nous avons diminué les paramètres à tester pour que ça prenne moins de temps
parameter_grid = {
    "vect__max_df": (0.6, 0.8),
    "vect__min_df": (5,6),
    "vect__norm": ("l1", "l2"),
}

random_search = GridSearchCV(
    scoring='f1',
    estimator=pipeline,
    param_grid=parameter_grid,
    n_jobs=2,
    verbose=1,
)

print("Performing grid search...")
print("Hyperparameters to be evaluated:")
pprint(parameter_grid)

# Recherche des meilleurs hyper-paramètres
t0 = time()
random_search.fit(train_df['fulltext'], train_df['category'])
print(f"Done in {time() - t0:.3f}s")

Performing grid search...
Hyperparameters to be evaluated:
{'vect__max_df': (0.6, 0.8), 'vect__min_df': (5, 6), 'vect__norm': ('l1', 'l2')}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Done in 15.318s


In [6]:
from sklearn.metrics import f1_score

# Affichage des résultats de la recherche des hyper-paramètres
print("Best parameters combination found:")
best_parameters = random_search.best_estimator_.get_params()
for param_name in sorted(parameter_grid.keys()):
    print(f"{param_name}: {best_parameters[param_name]}")

score = random_search.score(test_df['fulltext'], test_df['category'])
print(
    "F1-score of the best parameters using the inner CV of "
    f"the random search: {random_search.best_score_:.3f}"
)
print(f"F1-score on test set: {score:.3f}")

Best parameters combination found:
vect__max_df: 0.8
vect__min_df: 5
vect__norm: l2
F1-score of the best parameters using the inner CV of the random search: 0.935
F1-score on test set: 0.906


## Autres options et résultats obtenus
CountVectorizer n'a pas donné de très bons résultats, nous avons donc gardé TF-IDF.

Nous avons comparés ComplementNB et SGDClassifier. SGDClassifier a donné un score environ ~20% plus haut que ComplementNB.

In [7]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# On récupère le modèle qui a eu le meilleur f1-score
best_estimator = random_search.best_estimator_
pred = best_estimator.predict(test_df['fulltext'])
expected = test_df['category']

# Calcul des scores demandés
f1 = f1_score(pred, expected)
recall = recall_score(pred, expected)
precision = precision_score(pred, expected)

print('f1 :', f1)
print('recall :', recall)
print('precision :', precision)

f1 : 0.9064748201438849
recall : 0.9767441860465116
precision : 0.8456375838926175


### Résultats finaux
Au final, notre meilleur modèle obtient les scores suivants :
f1 : 0.906
recall : 0.977
precision : 0.846